<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lrstock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install FinMind==1.5.4

In [91]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from FinMind import plotting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [92]:
stock_id = '2330'
start_date = '2018-01-01'
end_date = '2023-08-10'
data_loader = DataLoader()
stock_data = data_loader.taiwan_stock_daily(stock_id, start_date, end_date)
stock_data = data_loader.feature.add_kline_institutional_investors(
    stock_data
)
stock_data = data_loader.feature.add_kline_margin_purchase_short_sale(
    stock_data
)
all_dates = stock_data['date'].values[1:]
selected_columns = ['Foreign_Investor_diff','Margin_Purchase_diff','Investment_Trust_diff', 'close' ]
filtered_data = stock_data[selected_columns]
numpy_array = filtered_data.values
#print(type(stock_data))
print(stock_data)
#print(type(numpy_array))
#print(numpy_array)


            date stock_id  Trading_Volume  Trading_money   open    max    min  \
0     2018-01-02     2330        18055269     4188555408  231.5  232.5  231.0   
1     2018-01-03     2330        31706091     7504382512  236.0  238.0  235.5   
2     2018-01-04     2330        29179613     6963192636  240.0  240.0  236.5   
3     2018-01-05     2330        23721255     5681934695  240.0  240.0  238.0   
4     2018-01-08     2330        21846692     5281823362  242.0  242.5  240.5   
...          ...      ...             ...            ...    ...    ...    ...   
1360  2023-08-04     2330        29320685    16266624892  556.0  560.0  552.0   
1361  2023-08-07     2330        15607870     8719529904  558.0  561.0  556.0   
1362  2023-08-08     2330        21167596    11708530913  558.0  558.0  551.0   
1363  2023-08-09     2330        15088059     8355524852  550.0  557.0  550.0   
1364  2023-08-10     2330        18884431    10418779108  552.0  554.0  550.0   

      close  spread  Tradin

In [95]:
X = numpy_array[:-1,:-1]  # 從第一行到倒數第二行的所有數據
y = numpy_array[1:, -1]  # 從第二行開始的所有行的最後一列（即隔天的close價格）

split_index = int(len(X) * 0.8)  # 80% 資料作為訓練集
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index:]
#normalization 效果竟然比standard效果好？
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# 縮放目標值
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
# 定義模型參數
W = tf.Variable(tf.random.normal([X_train_scaled.shape[1], 1], dtype=tf.float64), dtype=tf.float64, name="weight")
#b = tf.Variable(tf.random.normal([1], dtype=tf.float64), dtype=tf.float64, name="bias")
b = tf.Variable(tf.zeros([1], dtype=tf.float64), dtype=tf.float64, name="bias")

# 定義模型
def linear_regression(X):
    return tf.matmul(X, W) + b

# 定義損失
def mean_square(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - tf.cast(y_true, tf.float64)))
def mean_absolute_error(y_pred, y_true):
    return tf.reduce_mean(tf.abs(y_pred - tf.cast(y_true, tf.float64)))

# 定義優化器
#optimizer = tf.optimizers.SGD(learning_rate=0.1)
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# 訓練模型
n_epochs = 300
for epoch in range(n_epochs):
    with tf.GradientTape() as tape:
        y_pred = linear_regression(X_train_scaled)
        loss = mean_absolute_error(y_pred, y_train_scaled)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
# 評估模型
y_pred_test_scaled = linear_regression(X_test_scaled)
#print(X_test_scaled)
#print(y_pred_test_scaled)
#mse_test = mean_absolute_error(y_pred_test_scaled, y_test_scaled)
#print(f"Mean Squared Error on Scaled Test Data: {mse_test.numpy()}")
print(y_pred)
# 反縮放預測值回原始尺度
y_pred_test_original = scaler_y.inverse_transform(y_pred_test_scaled.numpy())
y_pred_train_original = scaler_y.inverse_transform(y_pred.numpy())
#mse_test_original = mean_squared_error(y_test, y_pred_test_original)
#print(f"Mean Squared Error on Original Test Data: {mse_test_original}")

#覺得怪怪的但還是做了
# 使用模型對縮放後的訓練數據進行預測
#y_pred_train_scaled = linear_regression(X_train_scaled)
#y_pred_train_original = scaler_y.inverse_transform(y_pred_train_scaled.numpy())

#print(y_pred_train_original)

tf.Tensor(
[[0.22704264]
 [0.30187037]
 [0.14067294]
 ...
 [0.44173251]
 [0.4236686 ]
 [0.31716656]], shape=(1091, 1), dtype=float64)


In [96]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='markers', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates, y=y_pred_train_original.flatten(), mode='markers', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='markers', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates, y=y_pred_test_original.flatten(), mode='markers', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

# 顯示圖形
fig.show()
